In [1]:
# Basic Imports
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import pickle
import re
import collections
import tables as tb
import csv

In [2]:
#Paths
in_file = '/projects/nikhil/ADNI_prediction/input_datasets/CS/ADNIMERGE.csv' #Original File

#in_file = '/projects/nikhil/ADNI_prediction/input_datasets/ADNI_Merge_filter_Longitudinal.csv' #Selected Columns
in_data = pd.read_csv(in_file,header=0)


In [3]:
#Some Defs
def save_dictionary(_dict,save_path):
    f = open(save_path, 'wb')
    pickle.dump(_dict, f)
    f.close()
    
def save_list(_list, save_path):
    ofile  = open(save_path, "wb")
    writer = csv.writer(ofile,quoting=csv.QUOTE_NONE,delimiter='\n')
    writer.writerow(_list)
    #for item in _list:
    #    writer.writerow([item])

    ofile.close()
    

In [4]:
#Extract various cohorts

#col_list = ['PTID','ADAS13']
col_list = ['PTID','DX_bl']
#col_list = ['PTID', 'DX_bl','DX']
#col_list = ['PTID']
cohort = 'ADNI2'
data_bl  = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'bl')][col_list]
data_m06 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm06')][col_list]
data_m12 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm12')][col_list]
data_m18 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm18')][col_list]
data_m24 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm24')][col_list]
data_m30 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm30')][col_list]
data_m36 = in_data[(in_data.ORIGPROT == cohort) & (in_data.COLPROT==cohort) & (in_data.VISCODE == 'm36')][col_list]

In [ ]:
data_mx = data_m12
data_bl_cn = data_bl[(data_bl.DX_bl == 'CN')]
data_bl_ad = data_bl[(data_bl.DX_bl == 'AD')]
data_bl_mci = data_bl[(data_bl.DX_bl == 'LMCI')]
data_mx_nl = data_mx[(data_mx.DX == 'NL')]
data_mx_mci = data_mx[(data_mx.DX == 'MCI')]
data_mx_ad = data_mx[(data_mx.DX == 'Dementia')]

data_mx_conv = data_mx[((data_mx.DX == 'MCI to Dementia') | (data_mx.DX == 'Dementia')) & (data_mx.DX_bl == 'LMCI')]
data_mx_conv2 = data_mx[(data_mx.DX == 'Dementia') & (data_mx.DX_bl == 'LMCI')]
data_mx_nonconv = data_mx[(data_mx.DX == 'MCI') & (data_mx.DX_bl == 'LMCI')]
print 'bl_(cn,ad,mci):{} {} {}, mx_(cn,ad,mci):{} {} {}, mx_(conv, conv2, nonconv): {} {} {}'.format(
                len(data_bl_cn),len(data_bl_ad),len(data_bl_mci), 
                len(data_mx_nl),len(data_mx_ad),len(data_mx_mci), 
                len(data_mx_conv), len(data_mx_conv2), len(data_mx_nonconv))

MCI_CONV_PTID_DX_dict = data_mx_conv.set_index('PTID')['DX'].to_dict()
MCI_NONCONV_PTID_DX_dict = data_mx_nonconv.set_index('PTID')['DX'].to_dict()

#print MCI_CONV_PTID_DX_dict.keys()
#print MCI_NONCONV_PTID_DX_dict.keys()

MCI_Dx_conversion_IDs = {'MCI_conv':MCI_CONV_PTID_DX_dict.keys(), 'MCI_nonconv':MCI_NONCONV_PTID_DX_dict.keys()}

#print MCI_Dx_conversion_IDs
save_dict = True
if save_dict: #Save data subsets
    save_dict_path = '/projects/nikhil/jlf/revision_data/adni_vol/MCI_1year_Dx_conversion_IDs.pkl'
    save_dictionary(MCI_Dx_conversion_IDs,save_dict_path)

In [ ]:
#sanity check with the HC vol subject list
ADNI1_HC_file = '/projects/nikhil/ADNI_prediction/input_datasets/HC/subject_HC_vol_dictionary_ADNI1_left.pkl'
pkl_file = open(ADNI1_HC_file, 'rb')
ADNI1_HC_data = pickle.load(pkl_file)
print len(ADNI1_HC_data)
HC_subs = BL_PTID_DX_bl_dict.keys()
print HC_subs[:5]

In [13]:
BL_dict = data_bl.set_index('PTID')['DX_bl'].to_dict()
print len(BL_dict)

CS_subs = BL_dict.keys()

#diff_subs = list(set(CS_subs) - set(HC_subs))

print CS_subs[:5]
#print len(diff_subs)

save_dict = True
if save_dict: #Save data subsets
    save_dict_path = '/projects/nikhil/ADNI_prediction/input_datasets/CS/ADNI2_BL_PTID_DX_bl_dict.pkl'
    save_dictionary(BL_dict,save_dict_path)
    
save_sub_list = False
if save_sub_list:
    sub_list = CS_subs
    save_ADNI2_sub_list_path = '/projects/nikhil/ADNI_prediction/input_datasets/CS/ADNIMERGE_ADNI2_BL_subject_list.txt'
    save_list(sub_list,save_ADNI2_sub_list_path)
    

787
['005_S_4707', '109_S_4380', '137_S_4303', '126_S_4458', '128_S_4671']


In [ ]:
#Rename Columns
data_bl.columns=['PTID','DX_bl','bl_score']
data_m06.columns=['PTID','DX_bl','m06_score']
data_m12.columns=['PTID','DX_bl','m12_score']
data_m18.columns=['PTID','DX_bl','m18_score']
data_m24.columns=['PTID','DX_bl','m24_score']
data_m30.columns=['PTID','DX_bl','m30_score']
data_m36.columns=['PTID','DX_bl','m36_score']

In [ ]:
#Compute Joins / Unions / Intersections
test_merge = pd.merge(data_bl, data_m06, on='PTID',how='inner')
test_merge = pd.merge(test_merge, data_m12, on='PTID',how='inner')
test_merge = pd.merge(test_merge, data_m18, on='PTID',how='inner')
test_merge = pd.merge(test_merge, data_m24, on='PTID',how='inner')
test_merge['m06_diff'] = test_merge['m06_score'] - test_merge['bl_score']
test_merge['m12_diff'] = test_merge['m12_score'] - test_merge['bl_score']
test_merge['m18_diff'] = test_merge['m18_score'] - test_merge['bl_score']
test_merge['m24_diff'] = test_merge['m24_score'] - test_merge['bl_score']


In [ ]:
#Plot configs
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 10)
plt.style.use('ggplot')

In [ ]:
#Plots
plot_datasets = ['bl_score','m06_score','m12_score','m18_score','m24_score','m06_diff','m12_diff','m18_diff','m24_diff']
n_timepts = 5
delta_timepts = n_timepts-1 
y_mean = []
y_err = []
dx='All'
for plt_ds in plot_datasets:    
    if dx == 'All':
        y = np.array(test_merge[plt_ds])
    else:
        y = np.array(test_merge[test_merge.DX_bl==dx][plt_ds])
    
    y_mean.append(np.mean(y))
    y_err.append( v(y))
    
#Plot the stats
font_small = 8
font_med = 16
font_large = 24

plt.subplot(1,2,1)
x = np.arange(n_timepts)
y_mean_trunc = y_mean[:n_timepts]
y_err_trunc = y_err[:n_timepts]
plt.errorbar(x,y_mean_trunc, yerr=y_err_trunc, linewidth=2, label='bl_dx: {}'.format(dx))
plt.xlim([-1,5])
plt.xticks(x,plot_datasets[:n_timepts],fontsize=font_med)
plt.xlabel('Timepoint',fontsize=font_med)
plt.ylabel('ADAS13 Score',fontsize=font_med)            
plt.legend(fontsize=font_med,loc=2)
plt.title('Longitudinal ADAS_score',fontsize=font_med)

plt.subplot(1,2,2)
x = np.arange(1,delta_timepts+1,1)
y_mean_trunc = y_mean[n_timepts:]
y_err_trunc = y_err[n_timepts:]
plt.errorbar(x,y_mean_trunc, yerr=y_err_trunc, linewidth=2, label='bl_dx: {}'.format(dx))
plt.xlim([-1,5])
plt.xticks(x,plot_datasets[n_timepts:],fontsize=font_med)
plt.xlabel('Timepoint',fontsize=font_med)
plt.ylabel('ADAS13 Score',fontsize=font_med)            
plt.legend(fontsize=font_med,loc=2)
plt.title('Longitudinal Change in ADAS_score',fontsize=font_med)


In [ ]:

plt.hist(test_merge['m06_diff'],bins=50,alpha=0.5,label='06')
plt.hist(test_merge['m12_diff'],bins=50,alpha=0.5,label='12')
#plt.hist(test_merge['m18_diff'],bins=50,alpha=0.5,label='18')
plt.hist(test_merge['m24_diff'],bins=50,alpha=0.5,label='24')
plt.legend()
plt.show()

In [ ]:
BL_PTID_DX_bl_dict

In [8]:
in_data[(in_data.PTID == '036_S_4740')]

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
